# Предварительная обработка данных

In [6]:
# ПРИВОДИМ К НУЖНОМУ ВИДУ БАЗУ ДАННЫХ ПО ИНСТРУМЕНТУ
# ПОХОДУ VOLUME ИЗМЕРЯЕТСЯ В ТЫСЯЧАХ ЛОТОВ, ПРОВЕРИТЬ
def format_values(name: pd.DataFrame) -> pd.DataFrame:
    """Функция для обработки базы данных свечей. Приводит базу к нужному виду 

    Args:
        name (pd.DataFrame): DataFrame со свечами по инструменту

    Returns:
        pd.DataFrame: База данных, приведенная к нужному для анализа виду
    """       
    name.rename(columns = {'open':"Open", "close":"Close", "high":"High", "low":"Low", "volume":"Volume"}, inplace = True) # ПЕРЕИМЕНОВАЛИ ДЛЯ ГРАФИКА ЧЕРЕЗ MPF. ЕСЛИ НЕ ИСПОЛЬЗОВАТЬ, ТО НЕ НАДО
    name['begin'] = name['begin'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) # ПРЕОБРАЗОВЫВАЕМ СТРОКУ В ДАТУ
    name = name[(name['begin'].dt.strftime('%H:%M') < "18:40") & (name['begin'].dt.strftime('%H:%M') >= "10:00")] # ОСТАВЛЯЕМ ТОЛЬКО ДНЕВНУЮ СЕССИЮ
    name.set_index("begin", inplace = True) # СТАВИМ ДАТУ КАК ИНДЕКС
    return name

In [7]:
sber = format_values(sber)
sber.head()

,Open,Close,High,Low,value,Volume
begin,,,,,,
2023-05-12 10:00:00,231.05,232.38,232.47,230.52,6.528957e+08,2819200
2023-05-12 10:01:00,232.38,232.00,232.38,231.53,3.482892e+08,1501950
2023-05-12 10:02:00,231.90,232.10,232.38,231.61,2.798282e+08,1205930
2023-05-12 10:03:00,232.08,231.50,232.09,230.90,2.537112e+08,1095840
2023-05-12 10:04:00,231.50,231.81,231.83,231.47,9.480026e+07,409130


In [8]:
# РАСЧЕТЫ НА ОСНОВЕ ОСНОВНЫХ ПОКАЗАТЕЛЕЙ. ВРОДЕ БЫ ВЕЗДЕ ДОЛИ, НО МОЖНО И ПРОЦЕНТЫ (ПРОВЕРИТЬ ЧТОБЫ ВСЕ БЫЛО ОДНООБРАЗНО)
def get_candle_characteristics(name: pd.DataFrame) -> pd.DataFrame:
    """Функция для расчета основных характеристик свечи по ее параметрам

    Args:
        name (pd.DataFrame): База данных со свечами по инструменту

    Returns:
        pd.DataFrame: База данных с посчитанными характеристиками свечей
    """    
    name["body"] = abs(name['Close'] - name["Open"])/(name["High"] - name["Low"]) # ОТНОШЕНИЕ ТЕЛА СВЕЧИ КО ВСЕЙ СВЕЧЕ
    name['body'].fillna(0, inplace = True) # ГДЕ HIGH = LOW ПИШЕМ 0
    name['colour'] = name.apply(lambda row: "green" if row['Close'] >= row["Open"] else "red", axis = 1) # КОДИРУЕМ ЦВЕТ СВЕЧИ
    name['weekday'] = name.index.day_name()
    #name['session'] = name.apply(lambda row: 'main' if row.name.time() < datetime.strptime('19:00', '%H:%M').time() else 'evening', axis = 1) # ТОРГОВАЯ СЕССИЯ
    name['change'] = (name['Close']- name['Open'])/name['Open'] # ИЗМЕНЕНИЕ ЦЕНЫ ЗА МИНУТУ
    name['span_up'] = name.apply(lambda row: (row['High'] - row['Open'])/row['Open'] if row["Open"] >= row['Close'] else (row['High'] - row['Close'])/row['Close'] , axis = 1) # ОТКЛОНЕНИЕ НАВЕРХ (ТЕНЬ)
    name['span_down'] = name.apply(lambda row: (row['Close'] - row['Low'])/row['Close'] if row["Open"] >= row['Close'] else (row['Open'] - row['Low'])/row['Open'] , axis = 1) # ОТКЛОНЕНИЕ ВНИЗ (ТЕНЬ)
    return name

In [9]:
sber = get_candle_characteristics(sber)
sber.head()

,Open,Close,High,Low,value,Volume,body,colour,weekday,change,span_up,span_down
begin,,,,,,,,,,,,
2023-05-12 10:00:00,231.05,232.38,232.47,230.52,6.528957e+08,2819200,0.682051,green,Friday,0.005756,0.000387,0.002294
2023-05-12 10:01:00,232.38,232.00,232.38,231.53,3.482892e+08,1501950,0.447059,red,Friday,-0.001635,0.000000,0.002026
2023-05-12 10:02:00,231.90,232.10,232.38,231.61,2.798282e+08,1205930,0.259740,green,Friday,0.000862,0.001206,0.001251
2023-05-12 10:03:00,232.08,231.50,232.09,230.90,2.537112e+08,1095840,0.487395,red,Friday,-0.002499,0.000043,0.002592
2023-05-12 10:04:00,231.50,231.81,231.83,231.47,9.480026e+07,409130,0.861111,green,Friday,0.001339,0.000086,0.000130


In [10]:
def get_TA(name: pd.DataFrame, n: int) -> pd.DataFrame:
    """Функция для расчета значений индикаторов технического анализа 

    Args:
        name (pd.DataFrame): База данных со свечами по инструменту
        n (int): Количество первых свечей для удаления (чтобы не было NaN у индикаторов и все они были посчитаны)

    Returns:
        pd.DataFrame: База данных с посчитанными значениями индикаторов
    """    
    name["rsi"] = talib.RSI(name['Close'], timeperiod = 14) # RSI
    macd, macdsignal, name['macdhist'] = talib.MACD(name['Close'], fastperiod = 12, slowperiod = 26, signalperiod = 9) # MACD
    name['slowk'], name['slowd'] = talib.STOCH(name['High'], name['Low'], name['Close'], fastk_period = 14, slowk_period = 1, slowk_matype = 0, slowd_period = 3, slowd_matype = 0) # STOCHASTIC
    name['adx'] = talib.ADX(name['High'], name['Low'], name['Close'], timeperiod = 14) #ADX
    name['atr'] = talib.ATR(name['High'], name['Low'], name['Close'], timeperiod = 14) #ATR
    name = name[n:]  # УДАЛЯЕМ ПЕРВЫЕ СТРОКИ ЧТОБЫ У ИНДИКАТОРОВ НЕ БЫЛО NaN. ПОДКОРРЕКТИРОВАТЬ В ЗАВИСИМОСТИ ОТ ПЕРИОДОВ ИНДИКАТОРОВ
    return name

In [11]:
sber = get_TA(sber, 33)
sber.head()

,Open,Close,High,Low,value,Volume,body,colour,weekday,change,span_up,span_down,rsi,macdhist,slowk,slowd,adx,atr
begin,,,,,,,,,,,,,,,,,,
2023-05-12 10:33:00,228.84,228.80,228.85,228.72,24568100.0,107390,0.307692,red,Friday,-0.000175,0.000044,0.000350,33.460241,0.003114,24.576271,20.762712,50.839494,0.493549
2023-05-12 10:34:00,228.77,228.93,229.18,228.73,42463547.5,185470,0.355556,green,Friday,0.000699,0.001092,0.000175,35.691689,0.021696,30.084746,26.836158,50.130020,0.490438
2023-05-12 10:35:00,228.93,228.94,229.20,228.90,29699548.2,129650,0.033333,green,Friday,0.000044,0.001136,0.000131,35.869849,0.038071,32.000000,28.887006,49.411119,0.476836
2023-05-12 10:36:00,228.94,228.86,229.00,228.50,32019311.9,139970,0.160000,red,Friday,-0.000349,0.000262,0.001573,35.033664,0.046375,29.090909,30.391885,49.231451,0.478490
2023-05-12 10:37:00,228.87,228.89,228.97,228.80,31303742.0,136780,0.117647,green,Friday,0.000087,0.000350,0.000306,35.639573,0.056098,37.640449,32.910453,49.064617,0.456455


In [12]:
# СВЕЧНЫЕ ПАТТЕРНЫ
def get_candle_patterns(name: pd.DataFrame) -> pd.DataFrame:
    """Функция для распознования свечных паттернов

    Args:
        name (pd.DataFrame): База данных со свечами по инструменту

    Returns:
        pd.DataFrame: База данных со свечными паттернами
    """    
    name['engulfing'] = talib.CDLENGULFING(name['Open'], name['High'], name['Low'], name['Close'])/100  # ENGULFING CANDLE
    name['doji'] = talib.CDLDOJI(name['Open'], name['High'], name['Low'], name['Close'])/100  # DOJI CANDLE
    name['hammer'] = talib.CDLHAMMER(name['Open'], name['High'], name['Low'], name['Close'])/100  # HAMMER CANDLE
    name['inv_hammer'] = talib.CDLINVERTEDHAMMER(name['Open'], name['High'], name['Low'], name['Close'])/100  # INVERTED HAMMER CANDLE
    name['shooting_star'] = talib.CDLSHOOTINGSTAR(name['Open'], name['High'], name['Low'], name['Close'])/100  # SHOOTING STAR CANDLE
    name['marubuzu'] = talib.CDLMARUBOZU(name['Open'], name['High'], name['Low'], name['Close'])/100  # MARUBOZU STAR CANDLE
    return name

In [13]:
sber = get_candle_patterns(sber)
sber.head()

,Open,Close,High,Low,value,Volume,body,colour,weekday,change,...,slowk,slowd,adx,atr,engulfing,doji,hammer,inv_hammer,shooting_star,marubuzu
begin,,,,,,,,,,,,,,,,,,,,,
2023-05-12 10:33:00,228.84,228.80,228.85,228.72,24568100.0,107390,0.307692,red,Friday,-0.000175,...,24.576271,20.762712,50.839494,0.493549,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-12 10:34:00,228.77,228.93,229.18,228.73,42463547.5,185470,0.355556,green,Friday,0.000699,...,30.084746,26.836158,50.130020,0.490438,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-12 10:35:00,228.93,228.94,229.20,228.90,29699548.2,129650,0.033333,green,Friday,0.000044,...,32.000000,28.887006,49.411119,0.476836,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-12 10:36:00,228.94,228.86,229.00,228.50,32019311.9,139970,0.160000,red,Friday,-0.000349,...,29.090909,30.391885,49.231451,0.478490,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-12 10:37:00,228.87,228.89,228.97,228.80,31303742.0,136780,0.117647,green,Friday,0.000087,...,37.640449,32.910453,49.064617,0.456455,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
def get_target_answers(name: pd.DataFrame) -> tuple: 
    """Функция, возвращающая цвет следующей свечи и записывающая его в базу данных со свечами по инструменту

    Args:
        name (pd.DataFrame): База данных со свечами по инструменту

    Returns:
        tuple: Кортеж, содержащий DataFrame с обновленными данными и массив с правильными ответами (цвет следующей свечи)
    """       
    y = name["change_all"][1:].values # ПОЛУЧАЕМ ВСЕ СВЕЧИ КРОМЕ ПЕРВОЙ
    name = name[:-1] # УДАЛЯЕМ ПОСЛЕДНЮЮ СТРОКУ, ДЛЯ НЕЕ НЕТ ПРОГНОЗА
    name['target'] = y # ЗАПИСЫВАЕМ ОТВЕТЫ
    return name, y

In [16]:
#sber, sber_y = get_target_answers(sber)
print(sber.shape)
sber.head()

(132402, 24)


Index(['Open', 'Close', 'High', 'Low', 'value', 'Volume', 'body', 'colour',
       'weekday', 'change', 'span_up', 'span_down', 'rsi', 'macdhist', 'slowk',
       'slowd', 'adx', 'atr', 'engulfing', 'doji', 'hammer', 'inv_hammer',
       'shooting_star', 'marubuzu'],
      dtype='object')

# ВСЕ ФУНКЦИИ В ОДНОМ МЕСТЕ


In [15]:
#get_candles("2023-05-12", "LKOH", 1, 'lkoh')
lukoil = pd.read_csv('lkoh.csv')
lukoil = format_values(lukoil)
lukoil = get_candle_characteristics(lukoil)
lukoil = get_TA(lukoil, 33)
lukoil = get_candle_patterns(lukoil)
#lukoil, lukoil_y = get_target_answers(lukoil)
print(lukoil.shape)
lukoil.tail()

(132402, 24)


,Open,Close,High,Low,value,Volume,body,colour,weekday,change,...,slowk,slowd,adx,atr,engulfing,doji,hammer,inv_hammer,shooting_star,marubuzu
begin,,,,,,,,,,,,,,,,,,,,,
2024-05-10 18:35:00,7716.5,7716.5,7717.0,7716.5,2924625.5,379,0.0,green,Friday,0.000000,...,0.000000,5.555556e+00,23.470194,0.852063,0.0,1.0,0.0,0.0,0.0,0.0
2024-05-10 18:36:00,7717.0,7716.0,7717.0,7716.0,1010859.5,131,1.0,red,Friday,-0.000130,...,0.000000,-5.750659e-12,25.671563,0.862630,-1.0,0.0,0.0,0.0,0.0,-1.0
2024-05-10 18:37:00,7716.0,7716.5,7716.5,7716.0,810205.0,105,1.0,green,Friday,0.000065,...,16.666667,5.555556e+00,27.715691,0.836727,0.0,0.0,0.0,0.0,0.0,1.0
2024-05-10 18:38:00,7716.5,7716.0,7716.5,7716.0,6311751.5,818,1.0,red,Friday,-0.000065,...,0.000000,5.555556e+00,29.613811,0.812676,0.0,0.0,0.0,0.0,0.0,-1.0
2024-05-10 18:39:00,7716.0,7716.5,7716.5,7716.0,2276222.0,295,1.0,green,Friday,0.000065,...,20.000000,1.222222e+01,31.376350,0.790342,0.0,0.0,0.0,0.0,0.0,1.0
